In [1]:
import os
import sqlite3
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from zipfile import ZipFile
from datatable import dt, f, by

# Funções Auxiliares

In [2]:
def get_zip_namelist(zip_path):
  files = ZipFile(zip_path).namelist()
  files = [f'{zip_path}/{f}'
    for f in files if 'dict' not in f]
  return files

In [3]:
def read_table(table_path, columns):
  cols_order = list(columns.values())
  columns[...] = None
  df = dt.fread(table_path, columns=columns)
  return df

In [4]:
def data_load(zip_path, cols, filter_func):
  files = get_zip_namelist(zip_path)
  dfs = list()
  for file_path in tqdm(files):
    df = read_table(file_path, cols)
    df = filter_func(df)
    dfs.append(df)
  df = dt.rbind(*dfs)
  return df

### SQL

In [5]:
def create_connection(db_file):
  conn = None
  try:
    conn = sqlite3.connect(db_file)
  except sqlite3.Error as e:
    print(e)
  return conn

In [6]:
def append_table_to_con(df, con, table):
  try:
    df.to_sql(
      name = table,
      con = con,
      if_exists = 'append', # append | replace
      index = False,
    )
  except Exception as excep:
    print(excep)
    print(df.columns)

In [7]:
# conn = create_connection('gestantes.db')
# with conn:
#   for df, table in zip([df_sinasc, df_cnes], ['sinasc', 'cnes']):
#     append_table_to_con(df.to_pandas(), conn, table)

# Fonte de Dados

Utilizamos algumas fontes para compor nossa base de dados de análise, abaixo descrevemos cada uma delas:

## CNES

### Descrição

Dados do Cadastro Nacional de Estabelecimentos de Saúde (CNES) reunidos pela Plataforma de Ciência de Dados aplicada à Saúde (PCDaS). Os dados foram obtidos junto ao DATASUS (Ministério da Saúde), tratados e enriquecidos seguindo uma metodologia própria de ETL da PCDaS, resultando em um dataset mensal com todos os registros de estabelecimentos de saúde contidos no CNES à partir de agosto de 2005. A atualização dos dados é realizada pelo DATASUS e a PCDaS extrai os registros mensais assim que disponibilizados. As colunas com nomes em MAIÚSCULO representam dados originais advindos do DATASUS e colunas com nomes iniciando em minúsculo representam dados resultantes de transformação ou enriquecimento.

### Selecionar

#### Colunas

In [8]:
cols_cnes = {
  'CNES': 'cnes', # Número nacional do estabelecimento de saúde
  'uf_SIGLA_UF': 'uf', # Sigla da unidade da federação do estabelecimento
  'mun_codigo_adotado': 'cod_mun', # Armazena o código atribuído ao município do estabelecimento, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo
  'def_esfera_a': 'esfera_adm', # Código da esfera administrativa
  'def_retencao': 'retencao_tributos', # Código de retenção de tributos
  'def_cod_ir': 'tipo_retenção', # Indica o tipo de retenção de tributos da mantenedora
  'def_natureza': 'natureza', # Código da natureza da organização
  'VINC_SUS': 'sus', # Vínculo com SUS: 1-Sim 0-Não
}

#### Linhas

In [9]:
def filter_cnes(df):
  cols = list(cols_cnes.values())[:-2]
  return df[(f['sus'] == 1), cols]

### Carregar dados

In [10]:
path_zip_cnes = f'{Path.home()}/Databases/CNES/CNES.zip'
output_cnes = path_zip_cnes.replace('zip', 'csv.gzip')
# df_cnes = data_load(path_zip_cnes, cols_cnes, filter_cnes)
# df_cnes.to_csv(output_cnes)
df_cnes = dt.fread(output_cnes)
df_cnes

,cnes,uf,cod_mun,esfera_adm,retencao_tributos,tipo_retenção,natureza
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2786052,SP,350410,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
1,2814714,SP,355030,PRIVADA,Estabelecimento PRIVADO LUCRATIVA,,Empresa Privada
2,2784769,SP,351880,ESTADUAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
3,2051001,SP,350440,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
4,8012806,SP,352390,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
5,2028476,SP,355030,ESTADUAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
6,2093731,SP,355030,MUNICIPAL,Estabelecimento PUBLICO,,"Administração Direta da Saúde (MS, SES, e SMS)"
7,2093766,SP,354090,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
8,2093871,SP,354090,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"


100%|██████████| 4374/4374 [05:40<00:00, 12.86it/s]

## SINASC

### Descrição

Dados do Sistema de Informações sobre Nascidos Vivos (SINASC) reunidos pela Plataforma de Ciência de Dados aplicada à Saúde (PCDaS). Os dados foram obtidos junto ao DATASUS (Ministério da Saúde), tratados e enriquecidos seguindo uma metodologia própria de ETL da PCDaS, resultando em um dataset anual com todos os registros das declarações de nascidos vivos contidas no SINASC à partir de 1996. A atualização dos dados é realizada pelo DATASUS e a PCDaS extrai os registros anuais assim que disponibilizados. As colunas com nomes em MAIÚSCULO representam dados originais advindos do DATASUS e colunas com nomes iniciando em minúsculo representam dados resultantes de transformação ou enriquecimento.

### Selecionar

#### Colunas

In [11]:
cols_sinasc = {
  'ano_nasc': 'ano',
  'CODESTAB': 'cnes',
  'nasc_codigo_adotado': 'hosp_municipio',
  'res_codigo_adotado': 'res_municipio',
  'nasc_RSAUDCOD': 'hosp_regiao_saude',
  'res_RSAUDCOD': 'res_regiao_saude',
  'def_parto': 'tipo_parto',
  'GRAVIDEZ': 'n_gestados',
  'CONSULTAS': 'n_pre_natal',
  'IDADEMAE': 'idade',
  'ESCMAE': 'nivel_escolaridade',
  'RACACORMAE': 'raca_cor',
  'def_raca_cor': 'nasc_raca_cor',
  'def_sexo': 'nasc_sexo',
  'PESO': 'nasc_peso',
  'APGAR1': 'nasc_apgar1',
  'APGAR5': 'nasc_apgar5',
  'LOCNASC': 'local_nasc',
}

#### Linhas

In [12]:
def filter_sinasc(df):
  cols = list(cols_sinasc.values())[:-2]
  cols = [c for c in cols if c in df.names]
  if 'cnes' in df.names:
    df_filtered = df[
      (
        (f['ano'] > 2009) & (f['ano'] < 2020)
      ) & (
        (f['idade'] > 9) & (f['idade'] < 50)
      ) & (
        (f['tipo_parto'] != 'Ignorado') # tipo parto
      ) & (
        (f['cnes'] != None)
      ) & (
        (f['local_nasc'] == 1) # Hospital
      ) & (
        (f['res_regiao_saude'] != 5301) # DF
      ) & (
        (f['res_regiao_saude'] != None)
      ) & (
        (f['hosp_regiao_saude'] != None)
      ) & (
        (f['res_municipio'] != None)
      ) & (
        (f['hosp_municipio'] != None)
      ), cols]
  else:
    df_filtered = dt.Frame()
  return df_filtered

### Carregar dados

In [13]:
path_zip_sinasc = f'{Path.home()}/Databases/SINASC/SINASC.zip'
output_sinasc = path_zip_sinasc.replace('zip', 'csv.gzip')
# df_sinasc = data_load(path_zip_sinasc, cols_sinasc, filter_sinasc)
# df_sinasc.to_csv(output_sinasc)
df_sinasc = dt.fread(output_sinasc)
df_sinasc

,ano,cnes,hosp_municipio,res_municipio,hosp_regiao_saude,res_regiao_saude,tipo_parto,n_gestados,n_pre_natal,idade,…,nasc_raca_cor,nasc_sexo,nasc_peso,nasc_apgar1,nasc_apgar5
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,2515598,110002,120040,1104,1201,Cesáreo,1,4,24,…,Branca,Masculino,3400,9,10
1,2010,3792595,110011,120040,1101,1201,Cesáreo,1,3,22,…,Parda,Masculino,3950,8,9
2,2010,2798484,110030,120040,1103,1201,Vaginal,1,4,20,…,Branca,Masculino,3550,8,9
3,2010,5701929,120001,120001,1201,1201,Vaginal,1,3,21,…,Parda,Feminino,3000,9,10
4,2010,5701929,120001,120001,1201,1201,Vaginal,1,3,31,…,Parda,Feminino,3000,9,10
5,2010,5701929,120001,120001,1201,1201,Vaginal,1,4,23,…,Parda,Masculino,3900,7,9
6,2010,5701929,120001,120001,1201,1201,Vaginal,1,4,26,…,Parda,Feminino,3250,7,10
7,2010,5701929,120001,120001,1201,1201,Vaginal,1,4,21,…,Parda,Masculino,3500,8,10
8,2010,5701929,120001,120001,1201,1201,Vaginal,1,4,20,…,Parda,Feminino,2900,8,9


100%|██████████| 675/675 [03:03<00:00,  3.69it/s]

## SIH

### Descrição

Dados do Sistema de Informações Hospitalares do SUS (SIHSUS) reunidos pela Plataforma de Ciência de Dados aplicada à Saúde (PCDaS). Os dados foram obtidos junto ao DATASUS (Ministério da Saúde), tratados e enriquecidos seguindo uma metodologia própria de ETL (processo de extração, transformação e carga dos dados) da PCDaS, resultando em um dataset mensal com todos os registros de internações hospitalares contidos no SIHSUS à partir de janeiro de 2008.

A atualização dos dados é realizada pelo DATASUS e a PCDaS extrai os registros mensais referentes aos arquivos `RD_UF_ANO_MÊS.dbc` (Reduzidos de AIH) assim que disponibilizados. As colunas com nomes em MAIÚSCULO representam dados originais advindos do DATASUS e colunas com nomes iniciando em minúsculo representam dados resultantes de transformação ou enriquecimento.

### Selecionar

#### Colunas

In [14]:
cols_sih = {
  'ano_internacao': 'ano',
  'CNES': 'cnes',
  'int_codigo_adotado': 'hosp_municipio',
  'res_codigo_adotado': 'res_municipio',
  'int_RSAUDCOD': 'hosp_regiao_saude',
  'res_RSAUDCOD': 'res_regiao_saude',
  'PROC_REA': 'tipo_parto',
  'def_idade_anos': 'idade',
}

#### Linhas

In [15]:
def filter_sih(df):
  cols = list(cols_sinasc.values())[:-1]
  cols = [c for c in cols if c in df.names]
  df_filtered = df[
    (
      (f['ano'] > 2009) & (f['ano'] < 2020)
    ) & (
      (f['idade'] > 9) & (f['idade'] < 50)
    ) & (
      (f['tipo_parto'] == 310010039) | (f['tipo_parto'] == 411010034)
    ) & (
      (f['cnes'] != None)
    ) & (
      (f['res_regiao_saude'] != 5301) # DF
    ) & (
      (f['res_regiao_saude'] != None)
    ) & (
      (f['hosp_regiao_saude'] != None)
    ) & (
      (f['res_municipio'] != None)
    ), cols]
  return df_filtered

### Carregar dados

In [17]:
path_zip_sih = '/Volumes/SanDisk/SIHSUS.zip'
output_sih = f'{Path.home()}/Databases/CNES/CNES.csv.gzip'
# df_sih = data_load(path_zip_sih, cols_sih, filter_sih)
# df_sih.to_csv(output_sih)
df_sih = dt.fread(output_sih)
df_sih

,cnes,uf,cod_mun,esfera_adm,retencao_tributos,tipo_retenção,natureza
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2786052,SP,350410,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
1,2814714,SP,355030,PRIVADA,Estabelecimento PRIVADO LUCRATIVA,,Empresa Privada
2,2784769,SP,351880,ESTADUAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
3,2051001,SP,350440,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
4,8012806,SP,352390,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
5,2028476,SP,355030,ESTADUAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
6,2093731,SP,355030,MUNICIPAL,Estabelecimento PUBLICO,,"Administração Direta da Saúde (MS, SES, e SMS)"
7,2093766,SP,354090,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"
8,2093871,SP,354090,MUNICIPAL,Estabelecimento PUBLICO,Estabelecimento PUBLICO,"Administração Direta da Saúde (MS, SES, e SMS)"


100%|██████████| 4400/4400 [24:46<00:00,  2.96it/s]